# **Многослойоный перцептрон** <br>


Для работы кода необходимо предоставить доступ к Google Disk, создать в корне папки Autocorrelation и Lables, загрузить в них набор данных и метки, а также создать в корне папку multilayer_perceptron для сохранения результатов обучения

Загрузка данных с Google Disk

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
!ls "/content/drive/My Drive/"

 2_5305779684380783213.docx   IMG_20180624_012413.jpg
 baza.pdf		      IMG_20180624_012420.jpg
'Colab Notebooks'	      IMG_20180624_012444.jpg
 DSCF3993.jpg		      IMG_20180624_012451.jpg
 DSCF4073.jpg		      IMG_20180624_012500.jpg
 DSCF4121.jpg		      IMG_20180624_012501.jpg
 DSCF4264.jpg		      IMG_20180624_013405.jpg
 DSCF4275.jpg		      IMG_20191218_123350.jpg
 DSCF4971.jpg		      IMG_20191218_123418.jpg
 IMG_0689.jpg		      IMG_20191218_124720.jpg
 IMG_0823.jpg		      IMG_20191218_130138.jpg
 IMG_0878.jpg		      Lektsia_1_2022.ppt
 IMG_0954.jpg		      Lektsia_2_2022.ppt
 IMG_20180623_215438.jpg      Lektsia_3_2022.ppt
 IMG_20180623_215449.jpg      otvety_1.docx
 IMG_20180623_220917.jpg      otvety_2.docx
 IMG_20180623_224551.jpg      otvety_3.docx
 IMG_20180623_224604.jpg      Transmetropolitan
 IMG_20180623_224606.jpg      ULA_6552.gdoc
 IMG_20180623_224923.jpg      ULA_6675.gdoc
 IMG_20180623_225647.jpg      ULA_6679.gdoc
 IMG_20180623_225943.jpg      ULA_6681.gdoc
 IMG_2018062

Подключение библиотек

In [16]:
import matplotlib
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Dense
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
from minisom import MiniSom
import pathlib
import pandas
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Преобразование набора данных в формат, удобный для обучения. Эта часть кода может быть выполнена один раз, поскольку в нем преобразованные данные сохраняются на Google Disk, после чего они могут быть загружены в уже готовом виде

In [17]:
au_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Autocorrelation/").glob('**/*')
AutocorPaths = [x for x in au_p if x.is_file()]
random.seed(40)
la_p = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Labels/").glob('**/*')
LabelsPaths = [x for x in la_p if x.is_file()]
print(len(AutocorPaths))
Paths = list(zip(AutocorPaths, LabelsPaths))
random.shuffle(Paths)
#AutocorPaths, LabelsPaths = zip(*temp)
#AutocorPaths, LabelsPaths = list(AutocorPaths), list(LabelsPaths)

data = []
labels = []


for autocorpath, labelspath in Paths:
  auto_file = open(autocorpath,'r')
  label_file = open(labelspath,'r') 

  for auto_line in auto_file:
    auto_temp = auto_line.split()
    autocor_sample = [float(item) for item in auto_temp]

    label_line = label_file.readline()
    label_temp = label_line.split()
    label_sample = [int(item) for item in label_temp]

    # Функция автокорреляции записывается столько раз, сколько меток ей сопоставлено
    for label in label_sample:
      one_hot_label = [0, 0, 0, 0, 0]
      one_hot_label[label-1] = 1
      labels.append(one_hot_label)
      data.append(autocor_sample)
  

  label_file.close()
  auto_file.close()
    

print(len(labels))

data = np.array(data)
labels = np.array(labels)

with open("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data saved")

with open("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels saved")

1
745
Data saved
Labels saved


Преобразование набора данных в обучающую и тестовую выборки

In [18]:
with open("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=40)

#(trainX, testX, train_labels_nums, test_labels_nums) = train_test_split(data, labels_nums,
#                                                  test_size=0.15,
#                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


Создание архитектуры нейронной сети

In [19]:
model = Sequential() 
model.add(Dense(30,input_shape=(30,), activation='sigmoid'))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(5, activation='softmax'))

INIT_LR = 0.01
opt = Adam(learning_rate=INIT_LR)

model.compile(loss="categorical_crossentropy",optimizer=opt,
              metrics=["categorical_accuracy"])

print ("Model compiled")

Model compiled


Обучение. Используется параметр checkpointer, позволяющий после обучения сохранить нейронную сеть, показавшую наилучший результат. Также может быть использован параметр early_stopping, останавливающий обучение, если в течение заданного количества эпох не наблюдалось в качестве сети

In [20]:
EPOCHS = 1000

# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet_{epoch:3d}_{categorical_accuracy:.3f}.model', verbose=1, save_freq=10)
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet_best.model', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/point.model', verbose=1)
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=200, verbose=1)


# model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])


H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=32, callbacks = [checkpointer])
# H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=32, callbacks = [early_stopping])
#H = model.fit(trainX, trainY, validation_data=(testX,testY), epochs=EPOCHS, batch_size=128)

print("Model trained")

Epoch 1/1000
 1/20 [>.............................] - ETA: 12s - loss: 1.5920 - categorical_accuracy: 0.3125
Epoch 1: val_loss improved from inf to 1.46633, saving model to /content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet_best.model
20/20 [==============================] - 2s 86ms/step - loss: 1.5695 - categorical_accuracy: 0.2923 - val_loss: 1.4663 - val_categorical_accuracy: 0.5357
Epoch 2/1000
19/20 [===========================>..] - ETA: 0s - loss: 1.3583 - categorical_accuracy: 0.5576
Epoch 2: val_loss improved from 1.46633 to 1.21687, saving model to /content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet_best.model
20/20 [==============================] - 1s 45ms/step - loss: 1.3574 - categorical_accuracy: 0.5545 - val_loss: 1.2169 - val_categorical_accuracy: 0.5804
Epoch 3/1000
 1/20 [>.............................] - ETA: 0s - loss: 1.2241 - categorical_accuracy: 0.5625
Epoch 3: val_loss improved from 1.21687 to 1.03228, saving model to /cont

Построение графика зависимости качества обучения сети от количества прошедших эпох. График, как и модель, полученная на последней эпохе, сохраняются в Google Disk в папке multilayer_perceptron. <br>
В случае использования при обучении параметра early_stopping для корректного построения графика необходимо раскомментировать первую строчку кода и закомментировать вторую

In [21]:
#N = np.arange(0, early_stopping.stopped_epoch+1)
N = np.arange(0, EPOCHS)
plt.clf()
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="vall_loss")
plt.plot(N, H.history["categorical_accuracy"], label="train_acc")
plt.plot(N, H.history["val_categorical_accuracy"], label="val_acc")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/")
plt.legend()
plt.savefig("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/Loss.png")
model.save("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet.model")

print("End")

End


Результаты обучения нейронной сети с использованием лучшей полученной модели. Выводятся результаты применения сети к тестовой выборке и отчет о качестве работы сети. В отчете представлены значения метрик precision, recall и f1-score для каждого из пяти типовых аварийных состояний судна, а также их средние значения для всех типов вместе

In [22]:
model = load_model("/content/drive/My Drive/Colab Notebooks/multilayer_perceptron/EasyNet_best.model")
predictions = model.predict (testX, batch_size = 32)
for i in predictions:
  print(i)
#print(predictions)

print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=("1 wreck type","2 wreck type","3 wreck type","4 wreck type","5 wreck type")))

4/4 [==============================] - 0s 2ms/step
[0.0020526  0.0006959  0.07169751 0.37585187 0.54970217]
[0.08493552 0.8130407  0.04979275 0.03795847 0.01427251]
[0.01749375 0.10813911 0.16348864 0.5218625  0.18901598]
[0.08788013 0.7877755  0.05647674 0.04946628 0.01840138]
[0.00267092 0.00211467 0.36219582 0.13811864 0.4948999 ]
[0.7645699  0.00079187 0.20151293 0.02382523 0.00930004]
[8.6624008e-03 4.0796923e-04 9.8264164e-01 6.4410032e-03 1.8468695e-03]
[2.3629346e-03 6.1578222e-04 9.8432797e-01 9.0473052e-03 3.6460587e-03]
[2.7186219e-03 6.9520233e-04 9.8996818e-01 4.8438753e-03 1.7742559e-03]
[1.0299358e-03 4.5904258e-04 1.9023933e-01 2.9976803e-01 5.0850362e-01]
[0.20081016 0.32091588 0.34170163 0.10990795 0.02666438]
[0.00141016 0.00208092 0.26035252 0.17000572 0.56615055]
[0.00193863 0.00439225 0.26032317 0.1939056  0.5394403 ]
[0.00349744 0.00105729 0.0064765  0.3625956  0.62637323]
[0.07888002 0.82963    0.04050123 0.03691753 0.01407116]
[9.0283567e-01 2.5175890e-04 6.954